In [ ]:
from brian2 import *
%matplotlib widget
prefs.codegen.target = 'numpy'

In [ ]:
start_scope()
tau = 10*ms
poisson_inp = PoissonGroup(100, rates=100*Hz)
neurons = NeuronGroup(1000, 'dv/dt = -v/tau : 1 (unless refractory)',
                      threshold='v > 1', reset='v = 0', refractory=3*ms,
                      method='exact')

syn = Synapses(poisson_inp, neurons, 'w : 1', on_pre='v += w')
syn.connect(p=0.2)
syn.w = 'rand()*0.1'

spike_mon = SpikeMonitor(neurons)
weight_mon = StateMonitor(syn, 'w', record=True, dt=100*ms, when='end')

@network_operation(dt=100*ms)
def update_weights(t):
    spike_trains = spike_mon.spike_trains()
    for idx in range(len(neurons)):
        spikes = spike_trains[idx]
        recent_spikes = spikes[spikes > t-100*ms]
        if len(recent_spikes) > 5:
            syn.w[:, idx] *= 0.9

run(1*second, report='text')

In [ ]:
max_idx = np.argmax(spike_mon.count)
plt.figure()
plt.plot(weight_mon.t/ms, weight_mon.w[syn.indices[:, max_idx]].T);